# Profiling BatchFlow code

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

This notebooks shows how to profile various parts of BatchFlow: namely, pipelines and models.

In [1]:
import sys

sys.path.append("../../..")
from batchflow import B, V, W
from batchflow.opensets import MNIST
from batchflow.models.torch import ResNet18

In [2]:
dataset = MNIST(bar=True)

100%|██████████| 8/8 [00:01<00:00,  5.00it/s]


To collect information about model training times (both on CPU and GPU), one must set `profile` option in the model configuration to `True`:

In [3]:
model_config = {
    'inputs/labels/classes': 10,
    'loss': 'ce',
    'profile': True,
}

In [4]:
pipeline = (dataset.train.p
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model('resnet', ResNet18, 'dynamic',
                            config=model_config)
                .train_model('resnet',
                             B.images, B.labels,
                             fetches='loss',
                             save_to=V('loss_history', mode='a'))
                )

To gather statistics about how long each action takes, we must set `profile` to `True` inside `run` call:

In [5]:
BATCH_SIZE = 64
N_ITERS = 50

pipeline.run(BATCH_SIZE, n_iters=N_ITERS, bar=True, profile='detailed')

  0%|                                                    | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|███████████████████████████████████████████| 50/50 [00:18<00:00,  2.76it/s]


# Pipeline profiling

In [6]:
pipeline.profile_info.head()

iter  \
action      id                                                         
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...     1   
            <built-in method sys.getrecursionlimit>::/opt/c...     1   
            <built-in method time.time>::../../../batchflow...     1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...     1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...     1   

                                                                total_time  \
action      id                                                               
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...    0.009695   
            <built-in method sys.getrecursionlimit>::/opt/c...    0.009695   
            <built-in method time.time>::../../../batchflow...    0.009695   
            <method 'match' of '_sre.SRE_Pattern' objects>:...    0.009695   
            <method 'match' of '_sre.SRE_Pattern' objects>:...    0.009695   

                                                                eval_time  \
action      id                                                              
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...   0.009661   
            <built-in method sys.getrecursionlimit>::/opt/c...   0.009661   
            <built-in method time.time>::../../../batchflow...   0.009661   
            <method 'match' of '_sre.SRE_Pattern' objects>:...   0.009661   
            <method 'match' of '_sre.SRE_Pattern' objects>:...   0.009661   

                                                                ncalls  \
action      id                                                           
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...      64   
            <built-in method sys.getrecursionlimit>::/opt/c...       1   
            <built-in method time.time>::../../../batchflow...       2   
            <method 'match' of '_sre.SRE_Pattern' objects>:...       1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...      66   

                                                                 tottime  \
action      id                                                             
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  0.000009   
            <built-in method sys.getrecursionlimit>::/opt/c...  0.000000   
            <built-in method time.time>::../../../batchflow...  0.000001   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000006   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000119   

                                                                 cumtime  \
action      id                                                             
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  0.000009   
            <built-in method sys.getrecursionlimit>::/opt/c...  0.000000   
            <built-in method time.time>::../../../batchflow...  0.000001   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000006   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000119   

                                                                  start_time  \
action      id                                                                 
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  1.627304e+09   
            <built-in method sys.getrecursionlimit>::/opt/c...  1.627304e+09   
            <built-in method time.time>::../../../batchflow...  1.627304e+09   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  1.627304e+09   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  1.627304e+09   

                                                                       batch_id  \
action      id                                                                    
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  139682942942456   
            <built-in method sys.getrecursionlimit>::/opt/c...  1396829429424

Note that there is a detailed information about exact methods that are called inside each of the actions. That is a lot of data which can give us precise understanding of parts of the code, that are our bottlenecks.

Columns of the `profile_info`:
- `action`, `iter`, `batch_id` and `start_time` are pretty self-explainable
- `id` allows to identify exact method with great details: it is a concatenation of `method_name`, `file_name`, `line_number` and `callee`
- `eval_time` is is a time taken by an action execution
- `total_time` is `eval_time` plus time of processing the profiling table at each iteration
- `action_time` is a time taken by an clear action execution without pipeline under-the-hood stuff
- `tottime` is a time taken by a method inside action
- `cumtime` is a time take by a method and all of the methods that are called inside this method


More often than not, though, we don't need such granularity. Pipeline method `show_profile_info` makes some handy aggregations:

**Note:** by default, results are sorted on `total_time` or `tottime`, depending on level of details.

In [7]:
# timings for each action
pipeline.show_profile_info(per_iter=False, detailed=False)

total_time                      eval_time                    
                      sum      mean       max        sum      mean       max
action                                                                      
train_model #2  14.424592  0.288492  4.403728  14.406723  0.288134  4.400935
to_array #0      0.450475  0.009009  0.016210   0.449449  0.008989  0.016191
multiply #1      0.343553  0.006871  0.010343   0.342571  0.006851  0.010316

In [8]:
# for each action show 2 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=False, detailed=True, sortby=('ncalls', 'max'), limit=2)

ncalls  \
                                                                      sum   
action         id                                                           
multiply #1    <built-in method builtins.isinstance>::../../.....    9600   
               <built-in method numpy.core._multiarray_umath.i...    6500   
to_array #0    <built-in method builtins.isinstance>::../../.....    9600   
               <built-in method numpy.core._multiarray_umath.i...    6500   
train_model #2 bw_parent::/opt/conda/lib/python3.6/site-packag...  595544   
               <method 'get' of 'dict' objects>::/opt/conda/li...   45216   

                                                                             \
                                                                       mean   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....    192.00   
               <built-in method numpy.core._multiarray_umath.i...    130.00   
to_array #0    <built-in method builtins.isinstance>::../../.....    192.00   
               <built-in method numpy.core._multiarray_umath.i...    130.00   
train_model #2 bw_parent::/opt/conda/lib/python3.6/site-packag...  11910.88   
               <method 'get' of 'dict' objects>::/opt/conda/li...    904.32   

                                                                          \
                                                                     max   
action         id                                                          
multiply #1    <built-in method builtins.isinstance>::../../.....    192   
               <built-in method numpy.core._multiarray_umath.i...    130   
to_array #0    <built-in method builtins.isinstance>::../../.....    192   
               <built-in method numpy.core._multiarray_umath.i...    130   
train_model #2 bw_parent::/opt/conda/lib/python3.6/site-packag...  15854   
               <method 'get' of 'dict' objects>::/opt/conda/li...  10083   

                                                                    tottime  \
                                                                        sum   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.001549   
               <built-in method numpy.core._multiarray_umath.i...  0.015209   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.002008   
               <built-in method numpy.core._multiarray_umath.i...  0.018474   
train_model #2 bw_parent::/opt/conda/lib/python3.6/site-packag...  0.135415   
               <method 'get' of 'dict' objects>::/opt/conda/li...  0.008735   

                                                                             \
                                                                       mean   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.000031   
               <built-in method numpy.core._multiarray_umath.i...  0.000304   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.000040   
               <built-in method numpy.core._multiarray_umath.i...  0.000369   
train_model #2 bw_parent::/opt/conda/lib/python3.6/site-packag...  0.002708   
               <method 'get' of 'dict' objects>::/opt/conda/li...  0.000175   

                                                                             \
                                                                        max   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.000053   
               <built-in method numpy.core._multiarray_umath.i...  0.000488   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.000084   
               <built-in method numpy.core._multiarray_umath.i...  0.000814   

In [9]:
# timings for each action for each iter
pipeline.show_profile_info(per_iter=True, detailed=False)

total_time  eval_time      batch_id
iter action                                             
1    train_model #2    4.403728   4.400935  1.396829e+14
     to_array #0       0.009695   0.009661  1.396829e+14
     multiply #1       0.007570   0.007544  1.396829e+14
2    train_model #2    0.208846   0.208776  1.396829e+14
     to_array #0       0.008315   0.008297  1.396829e+14
...                         ...        ...           ...
49   to_array #0       0.009335   0.009314  1.396821e+14
     multiply #1       0.005326   0.005313  1.396821e+14
50   train_model #2    0.146019   0.145514  1.396821e+14
     to_array #0       0.007550   0.007525  1.396821e+14
     multiply #1       0.005421   0.005405  1.396821e+14

[150 rows x 3 columns]

In [10]:
# for each iter each action show 3 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=True, detailed=True, sortby='tottime', limit=3)

ncalls  \
iter action         id                                                           
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....      64   
                    inner::../../../batchflow/batchflow/utils_rando...      64   
                    multiply::../../../batchflow/batchflow/batch.py...      64   
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....      64   
                    inner::../../../batchflow/batchflow/utils_rando...      64   
...                                                                        ...   
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...      64   
                    <built-in method numpy.array>::../../../batchfl...      64   
     train_model #2 parse_legacy_records::/opt/conda/lib/python3.6/...       1   
                    <method 'run_backward' of 'torch._C._EngineBase...       1   
                    __init__::/opt/conda/lib/python3.6/site-package...    2561   

                                                                         tottime  \
iter action         id                                                             
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....  0.001118   
                    inner::../../../batchflow/batchflow/utils_rando...  0.000891   
                    multiply::../../../batchflow/batchflow/batch.py...  0.000450   
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....  0.001375   
                    inner::../../../batchflow/batchflow/utils_rando...  0.001026   
...                                                                          ...   
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...  0.000696   
                    <built-in method numpy.array>::../../../batchfl...  0.000535   
     train_model #2 parse_legacy_records::/opt/conda/lib/python3.6/...  0.040087   
                    <method 'run_backward' of 'torch._C._EngineBase...  0.015765   
                    __init__::/opt/conda/lib/python3.6/site-package...  0.005153   

                                                                         cumtime  
iter action         id                                                            
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....  0.001460  
                    inner::../../../batchflow/batchflow/utils_rando...  0.001544  
                    multiply::../../../batchflow/batchflow/batch.py...  0.000629  
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....  0.001769  
                    inner::../../../batchflow/batchflow/utils_rando...  0.001827  
...                                                                          ...  
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...  0.001281  
                    <built-in method numpy.array>::../../../batchfl...  0.001483  
     train_model #2 parse_legacy_records::/opt/conda/lib/python3.6/...  0.067863  
                    <method 'run_backward' of 'torch._C._EngineBase...  0.015765  
                    __init__::/opt/conda/lib/python3.6/site-package...  0.005845  

[450 rows x 3 columns]

# Model profiling

In [11]:
model = pipeline.m('resnet')

There is an `info` property that, unsurprisingly, shows a lot of interesting details regarding model itself or the training process:

In [12]:
print(model.info())


##### Config:
{'amp': True,
 'benchmark': True,
 'body': {'encoder': {'blocks': {'base': <class 'batchflow.models.torch.blocks.ResBlock'>,
                                 'bottleneck': False,
                                 'downsample': [False, True, True, True],
                                 'filters': [64, 128, 256, 512],
                                 'layout': 'cnacn',
                                 'n_reps': [2, 2, 2, 2],
                                 'se': False},
                      'downsample': {'layout': 'p',
                                     'pool_size': 2,
                                     'pool_strides': 2},
                      'num_stages': 4,
                      'order': ['skip', 'block']}},
 'common': {'data_format': 'channels_first'},
 'decay': None,
 'device': None,
 'head': {'classes': 10,
          'dropout_rate': 0.4,
          'filters': 10,
          'layout': 'Vdf',
          'target_shape': (64,),
          'units': 10},
 'initial_bloc

As with pipeline, there is a `profile_info` attribute, as well as `show_profile_info` method. Depending on type of the used device (`CPU` or `GPU`)

In [13]:
# one row for every operation inside model; limit at 5 rows
model.show_profile_info(per_iter=False, limit=5)

ncalls           CUDA_cumtime  \
                                          sum  mean max          sum   
name                                                                   
Optimizer.step#Adam.step                   49   1.0   1     0.708343   
aten::conv2d                             2000  40.0  40     0.481064   
CudnnConvolutionBackward                 1000  20.0  20     0.385295   
aten::cudnn_convolution_backward         1000  20.0  20     0.372299   
aten::cudnn_convolution_backward_input    950  19.0  19     0.227744   

                                                           CUDA_cumtime_avg  \
                                            mean       max              sum   
name                                                                          
Optimizer.step#Adam.step                0.014456  0.018066         0.708343   
aten::conv2d                            0.009621  0.127704         0.012027   
CudnnConvolutionBackward                0.007706  0.114099         0.019265   
aten::cudnn_convolution_backward        0.007446  0.113787         0.018615   
aten::cudnn_convolution_backward_input  0.004555  0.057634         0.011987   

                                                            
                                            mean       max  
name                                                        
Optimizer.step#Adam.step                0.014456  0.018066  
aten::conv2d                            0.000241  0.003193  
CudnnConvolutionBackward                0.000385  0.005705  
aten::cudnn_convolution_backward        0.000372  0.005689  
aten::cudnn_convolution_backward_input  0.000240  0.003033

In [14]:
# for each iteration show 3 of the slowest operations
model.show_profile_info(per_iter=True, limit=3)

ncalls  CUDA_cumtime  CUDA_cumtime_avg
iter name                                                                    
0    aten::conv2d                          40      0.127704          0.003193
     CudnnConvolutionBackward              20      0.114099          0.005705
     aten::cudnn_convolution_backward      20      0.113787          0.005689
1    Optimizer.step#Adam.step               1      0.016724          0.016724
     CudnnConvolutionBackward              20      0.007106          0.000355
...                                       ...           ...               ...
48   CudnnConvolutionBackward              20      0.006753          0.000338
     aten::cudnn_convolution_backward      20      0.006666          0.000333
49   Optimizer.step#Adam.step               1      0.014673          0.014673
     aten::conv2d                          40      0.006112          0.000153
     aten::mul_                           124      0.006030          0.000049

[150 rows x 3 columns]